In [179]:
# !pip3 install googlesearch-python
# !pip3 install tld
# !pip3 install SPF2IP
# !pip install install tldextract

In [180]:
import pandas as pd
import numpy as np
from tld import get_tld, is_tld
import re
import bz2
import pickle
from googlesearch import search
from SPF2IP import SPF2IP
from urllib.parse import urlparse, parse_qs
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

In [181]:
# Config sys path and display
import sys
import warnings
import pandas as pd
sys.path.append('../')
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("display.width", 500)

from fraud.data.make_dataset import *

In [182]:
# Download and load dataset 
download_path = "../data/raw"
origin_data = download_and_load(download_path)
new_data = download_and_load_new(download_path)
new_data.head(5)

Skipping, found downloaded files in "../data/raw\dataset-phishing-domain-detection-cybersecurity" (use force=True to force download)


Skipping, found downloaded files in "../data/raw\malicious-urls-dataset" (use force=True to force download)


,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [183]:
# # Data Preprocessing and visualize data before training
# upload_path = "../data/processed"
# data = upload_new(preprocess(data), upload_path)
# pre_plot(data)

In [184]:
origin_data.columns.tolist()

['qty_dot_url',
 'qty_hyphen_url',
 'qty_underline_url',
 'qty_slash_url',
 'qty_questionmark_url',
 'qty_equal_url',
 'qty_at_url',
 'qty_and_url',
 'qty_exclamation_url',
 'qty_space_url',
 'qty_tilde_url',
 'qty_comma_url',
 'qty_plus_url',
 'qty_asterisk_url',
 'qty_hashtag_url',
 'qty_dollar_url',
 'qty_percent_url',
 'qty_tld_url',
 'length_url',
 'qty_dot_domain',
 'qty_hyphen_domain',
 'qty_underline_domain',
 'qty_slash_domain',
 'qty_questionmark_domain',
 'qty_equal_domain',
 'qty_at_domain',
 'qty_and_domain',
 'qty_exclamation_domain',
 'qty_space_domain',
 'qty_tilde_domain',
 'qty_comma_domain',
 'qty_plus_domain',
 'qty_asterisk_domain',
 'qty_hashtag_domain',
 'qty_dollar_domain',
 'qty_percent_domain',
 'qty_vowels_domain',
 'domain_length',
 'domain_in_ip',
 'server_client_domain',
 'qty_dot_directory',
 'qty_hyphen_directory',
 'qty_underline_directory',
 'qty_slash_directory',
 'qty_questionmark_directory',
 'qty_equal_directory',
 'qty_at_directory',
 'qty_and_dir

In [185]:
#df_phish = df.copy()
df_phish = new_data.loc[(new_data['type'] == 'benign') | (new_data['type'] == 'phishing')]
df_phish[['type']].value_counts()


type    
benign      428103
phishing     94111
Name: count, dtype: int64

In [186]:
df_phish.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign


## Updated - functions

In [187]:
x = "www.ww.google.com.sg"
os.path.basename(urlparse(x).path)

'www.ww.google.com.sg'

In [188]:
from urllib.parse import urlparse, unquote
x = "www.ww.google.com.sg/dir/file.sg"
urlparse(x)

ParseResult(scheme='', netloc='', path='www.ww.google.com.sg/dir/file.sg', params='', query='', fragment='')

In [189]:
urlparse(x).netloc + urlparse(x).path

'www.ww.google.com.sg/dir/file.sg'

In [190]:
x="games.teamxbox.com/s"
x.split("/")

['games.teamxbox.com', 's']

In [191]:
x=""
x.split("/")

['']

In [192]:
x = "www.ww.google.com.sg/dir/file.htm?val=1"
# tldextract.extract(x)
urlparse(x)

ParseResult(scheme='', netloc='', path='www.ww.google.com.sg/dir/file.htm', params='', query='val=1', fragment='')

In [193]:
lst = [1,2]
lst[1:]

[2]

In [194]:
"""
normal: (scheme//)domain/dir(/file)
file_length 0: 
    abc or abc/
special:
    
    
    
file_end: html, htm, phtml, shtml, asp, php, x, 
dir?param

remove training white space and quotation
    ie. 'www.armchairempire.com/Reviews/PC\%20Games/undying.htm'

remove scheme
    ie.

recognise file and dir
    ie. www.gameindustry.com/review/item.asp?id=613
    ie. www.1up.com/do/gameOverview?cId=3159391
        www.1up.com/do/previewPage?cId=3150301&amp;did=1
        www.gameindustry.com/review/dir/
        www.gameindustry.com/review/dir
    ie. www.realmofgaming.com/?page=ps2cheats/bloodrayne2.php&amp;type=cheats

decode
    ie. 'www.armchairempire.com/Reviews/PC\%20Games/undying.htm'
"""


"\nnormal: (scheme//)domain/dir(/file)\nfile_length 0: \n    abc or abc/\nspecial:\n    \n    \n    \nfile_end: html, htm, phtml, shtml, asp, php, x, \ndir?param\n\nremove training white space and quotation\n    ie. 'www.armchairempire.com/Reviews/PC\\%20Games/undying.htm'\n\nremove scheme\n    ie.\n\nrecognise file and dir\n    ie. www.gameindustry.com/review/item.asp?id=613\n    ie. www.1up.com/do/gameOverview?cId=3159391\n        www.1up.com/do/previewPage?cId=3150301&amp;did=1\n        www.gameindustry.com/review/dir/\n        www.gameindustry.com/review/dir\n    ie. www.realmofgaming.com/?page=ps2cheats/bloodrayne2.php&amp;type=cheats\n\ndecode\n    ie. 'www.armchairempire.com/Reviews/PC\\%20Games/undying.htm'\n"

In [195]:
def remove_trailing(url):
    url.strip().strip("'").strip()

def remove_scheme(url):
    return urlparse(url).netloc + urlparse(url).path + urlparse(url).params + urlparse(url).query + urlparse(url).fragment

def decode(url):
    return unquote(url)

def parse_dir(url):
    url_split = url.split("/")
    if len(url_split) <= 1:
        return ""
    elif len(url_split) == 2:
        return url_split[-1]
    else:
        return "/".join(url_split[1:-1])
    
def parse_file(url):
    url_split = url.split("/")
    if len(url_split) <= 1:
        return ""
    else: # maybe last maybe no, could be dir?params
        file_params = url_split[-1]
        if "?" in file_params:
            file = file_params.split("?")[0]
        else:
            file = file_params
    return file

def parse_params(url):
    return urlparse(url).query

In [196]:
from urllib.parse import urlparse, parse_qs, urlencode

from urllib.parse import urlparse, parse_qsl, urlencode, unquote_plus

def split_url(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract components
    domain = parsed_url.netloc
    path = parsed_url.path
    query = parsed_url.query

    # Decode URL-encoded parameters
    decoded_query = unquote_plus(query)

    # Parse query parameters
    parsed_query = parse_qsl(decoded_query)
    params_string = urlencode(parsed_query, doseq=True)

    # If there's no path, treat the URL as just a domain
    if not path:
        return {
            'domain': domain,
            'directory': None,
            'file': None,
            'params': params_string
        }

    # Split path into directory and file
    path_parts = path.split('/')
    directory = '/'.join(path_parts[:-1])
    file = path_parts[-1]

    return {
        'domain': domain,
        'directory': directory or None,
        'file': file or None,
        'params': params_string
    }
# Example URL
url = "br-icloud.com.br"

# Split the URL
url_parts = split_url(url)

# Print the parts
print("Domain:", url_parts['domain'])
print("Directory:", url_parts['directory'])
print("File:", url_parts['file'])
print("Parameters:", url_parts['params'])

def parse_xiangyu(df_phish):
    df_parts = df_phish['url'].apply(split_url)

# Create a new DataFrame with the parts as columns
    df_parts = pd.DataFrame(df_parts.tolist())

# Concatenate the original DataFrame with the new parts DataFrame
    df_concat = pd.concat([df_phish, df_parts], axis=1)

# Drop the original 'url' column if desired
# df_final = df_final.drop('url', axis=1)

 # Add length-related columns
    df_concat['url'] = df_concat['url'].fillna('')
    df_concat['domain'] = df_concat['domain'].fillna('')
    df_concat['directory'] = df_concat['directory'].fillna('')
    df_concat['file'] = df_concat['file'].fillna('')
    df_concat['params'] = df_concat['params'].fillna('')

    df_concat.loc[:, 'length_url'] = df_concat['url'].str.len().astype(int)
    df_concat.loc[:, 'domain_length'] = df_concat['domain'].str.len().astype(int)
    df_concat.loc[:, 'directory_length'] = df_concat['directory'].str.len().astype(int)
    df_concat.loc[:, 'file_length'] = df_concat['file'].str.len().astype(int)
    df_concat.loc[:, 'params_length'] = df_concat['params'].str.len().astype(int)
    return df_concat



Domain: 
Directory: None
File: br-icloud.com.br
Parameters: 


In [197]:

import tldextract
import os
def process_new_url(df):
    # Remove trailing space and quotation
    df['url'] = df['url'].apply(lambda x: remove_trailing(x))
    
    # Remove scheme
    df['url'] = df['url'].apply(lambda x: remove_scheme(x))
    
    # Decode
    df['url'] = df['url'].apply(lambda x: decode(x))
    
    # Extract domain
    try:
        df['domain'] = df['url'].apply(lambda x: tldextract.extract(x).domain + '.' + tldextract.extract(x).suffix)
    except:
        df.loc[:, 'domain'] = None

    # Extract directory
    # df['directory'] = df['url'].apply(lambda x: os.path.dirname(urlparse(x).path))
    df['directory'] = df['domain'].apply(lambda x: parse_dir(x))

    # Extract file
    # df['file'] = df['url'].apply(lambda x: os.path.basename(urlparse(x).path))
    df['file'] = df['domain'].apply(lambda x: parse_file(x))
    
    # Extract params
    df['params'] = df['domain'].apply(lambda x: parse_params(x))
    
    # Add length-related columns
    df.loc[:, 'length_url'] = df['url'].str.len()
    df.loc[:, 'domain_length'] = df['domain'].str.len()
    df.loc[:, 'directory_length'] = df['directory'].str.len()
    df.loc[:, 'file_length'] = df['file'].str.len()
    df.loc[:, 'params_length'] = df['params'].str.len()

    return df


In [198]:
df_phish = parse_xiangyu(df_phish)
df_phish.head()

,url,type,domain,directory,file,params,length_url,domain_length,directory_length,file_length,params_length
0,br-icloud.com.br,phishing,,,br-icloud.com.br,,16,0,0,16,0
1,mp3raid.com/music/krizz_kaliko.html,benign,,mp3raid.com/music,krizz_kaliko.html,,35,0,17,17,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,,bopsecrets.org/rexroth/cr,1.htm,,31,0,25,5,0
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,,yourbittorrent.com,,q=anthony-hamilton-soulife,118,0,18,0,26
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,,allmusic.com/album,crazy-from-the-heat-r16990,,45,0,18,26,0


### Reformat url dataset

In [199]:
# tls_ssl_certificate

In [200]:
def special_chars_qty(df):
    vowels=['a','e','i','o','u']
    features = {'at':'@', 'questionmark':'?', 'underline':'_', 'hyphen':'-', 'equal':'=', 'dot':'.', 
            'hashtag':'#', 'percent':'%', 'plus':'+', 'dollar':'$', 'exclamation':'!', 'asterisk':'*', 
            'comma':',', 'slash':'/', 'space':' ', 'tilde':'~','and':'&'}
    cols=['url','domain','params','directory','file']

    # add quantity of special characters for all cols
    for i in range(len(cols)):
        for key, value in features.items():
                df.loc[:, "qty_" + key + '_'+ cols[i]] = df_phish.loc[:, cols[i]].apply(lambda x: x.count(value) if x else -1)

    # add vowel qtr for domain
    df.loc[:,"qty_vowels_domain"] = df.loc[:,'domain'].apply(lambda x: 0 if x is None else sum(char.lower() in vowels for char in x))

    return df   


def shortened(url):
    match = re.search(
                      'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0 


def check_google_index(url):
    site = search(url, 3)
    return 1 if site else 0


# check if email in url
def check_email(url):
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
    if(re.fullmatch(regex, url)):
        return 1
    else:
        return 0


# check if domain has spf record
def check_spf(dom):
    try:
        spf_records = SPF2IP().query(dom)
        if spf_records:
            return 1
        else:
            return 0
    except:
        return 0


# check the quantity of tlds in url
def count_tlds(url):
    # Regular expression pattern to extract TLDs from a URL
    tld_pattern = r'\.([a-zA-Z]{2,})$'
    
    # Find all matches of the TLD pattern in the URL
    tlds = re.findall(tld_pattern, url)
    
    # Return the count of unique TLDs
    return len(set(tlds))


# check the number of params in url
def count_params(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the query parameters
    query_params = parsed_url.query
    
    # Parse the query parameters
    parsed_query_params = parse_qs(query_params)
    
    # Count the number of parameters
    num_params = len(parsed_query_params)
    
    return num_params


# use this to complete preprocessing of new dataset 
def reformat_df(df):
     
    df_new = special_chars_qty(df)
    df_new.loc[:,'url_shortened'] = df_new.loc[:,'url'].apply(lambda x: shortened(x))

    # add the quantity of params in url
    df_new.loc[:,'qty_params'] = df_new.loc[:,'url'].apply(lambda x: count_params(x))

    # check if google index is available for url & domain
    df_new.loc[:,'url_google_index'] = df_new.loc[:,'url'].apply(lambda i: check_google_index(i))
    df_new.loc[:,'domain_google_index'] = df_new.loc[:,'domain'].apply(lambda i: check_google_index(i)) 

    # check if email is in utl
    df_new.loc[:,'email_in_url'] = df_new.loc[:,'url'].apply(lambda i: check_email(i))

    # check if domain has spf record
    df_new.loc[:,'domain_spf'] = df_new.loc[:,'domain'].apply(check_spf)

    # check qty of tld in url
    df_new.loc[:,'qty_tld_url']=df_new.loc[:,'url'].apply(count_tlds)
    
    # Check if TLD is present in params column and return 1 or -1
    df_new['tld_present_params'] = df_new.apply(lambda row: 1 if tldextract.extract(row['params']).suffix in row['domain'] else -1, axis=1)

    # Create a mapping dictionary
    label_mapping = {'benign': 0, 'phishing': 1}
    df_new['phishing'] = df_new['type'].map(label_mapping)
    
    return df_new
    
    

In [201]:
# takes <60s
df_res = reformat_df(df_phish)
df_res.info()
df_res.head()

<class 'pandas.core.frame.DataFrame'>
Index: 631224 entries, 0 to 520325
Columns: 106 entries, url to phishing
dtypes: float64(1), int32(5), int64(94), object(6)
memory usage: 503.3+ MB


,url,type,domain,directory,file,params,length_url,domain_length,directory_length,file_length,params_length,qty_at_url,qty_questionmark_url,qty_underline_url,qty_hyphen_url,qty_equal_url,qty_dot_url,qty_hashtag_url,qty_percent_url,qty_plus_url,qty_dollar_url,qty_exclamation_url,qty_asterisk_url,qty_comma_url,qty_slash_url,qty_space_url,qty_tilde_url,qty_and_url,qty_at_domain,qty_questionmark_domain,qty_underline_domain,qty_hyphen_domain,qty_equal_domain,qty_dot_domain,qty_hashtag_domain,qty_percent_domain,qty_plus_domain,qty_dollar_domain,qty_exclamation_domain,qty_asterisk_domain,qty_comma_domain,qty_slash_domain,qty_space_domain,qty_tilde_domain,qty_and_domain,qty_at_params,qty_questionmark_params,qty_underline_params,qty_hyphen_params,qty_equal_params,qty_dot_params,qty_hashtag_params,qty_percent_params,qty_plus_params,qty_dollar_params,qty_exclamation_params,qty_asterisk_params,qty_comma_params,qty_slash_params,qty_space_params,qty_tilde_params,qty_and_params,qty_at_directory,qty_questionmark_directory,qty_underline_directory,qty_hyphen_directory,qty_equal_directory,qty_dot_directory,qty_hashtag_directory,qty_percent_directory,qty_plus_directory,qty_dollar_directory,qty_exclamation_directory,qty_asterisk_directory,qty_comma_directory,qty_slash_directory,qty_space_directory,qty_tilde_directory,qty_and_directory,qty_at_file,qty_questionmark_file,qty_underline_file,qty_hyphen_file,qty_equal_file,qty_dot_file,qty_hashtag_file,qty_percent_file,qty_plus_file,qty_dollar_file,qty_exclamation_file,qty_asterisk_file,qty_comma_file,qty_slash_file,qty_space_file,qty_tilde_file,qty_and_file,qty_vowels_domain,url_shortened,qty_params,url_google_index,domain_google_index,email_in_url,domain_spf,qty_tld_url,tld_present_params,phishing
0,br-icloud.com.br,phishing,,,br-icloud.com.br,,16,0,0,16,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1.000
1,mp3raid.com/music/krizz_kaliko.html,benign,,mp3raid.com/music,krizz_kaliko.html,,35,0,17,17,0,0,0,1,0,0,2,0,0,0,0,0,0,0,2,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0.000
2,bopsecrets.org/rexroth/cr/1.htm,benign,,bopsecrets.org/rexroth/cr,1.htm,,31,0,25,5,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0.000
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,,yourbittorrent.com,,q=anthony-hamilton-soulife,118,0,18,0,26,0,0,0,16,0,2,0,0,0,0,0,0,0,5,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,1,0,0,1,1,0.000
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,,allmusic.com/album,crazy-from-the-heat-r16990,,45,0,18,26,0,0,0,1,1,0,2,0,0,0,0,0,0,0,6,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0.000


In [202]:
# count how many -1 there are for each column
def count_minus_one(df):
    counts = {}
    for col in df.columns:
        # Count the occurrences of -1 in the column
        count_minus_one = (df[col] == -1).sum()
        
        # Store the count in the dictionary
        counts[col] = count_minus_one
    
    return counts


In [203]:
#take too long
'''
def count_redirects(url):
    try:
        response = requests.get(url, allow_redirects=True)
        num_redirects = len(response.history)
        return num_redirects
    except requests.exceptions.RequestException:
        # If an exception occurs during the request (e.g., invalid URL), return -1
        return -1

df_res['qty_redirects'] = df_res.loc[:,'url'].apply(count_redirects)
'''

"\ndef count_redirects(url):\n    try:\n        response = requests.get(url, allow_redirects=True)\n        num_redirects = len(response.history)\n        return num_redirects\n    except requests.exceptions.RequestException:\n        # If an exception occurs during the request (e.g., invalid URL), return -1\n        return -1\n\ndf_res['qty_redirects'] = df_res.loc[:,'url'].apply(count_redirects)\n"

In [204]:
# Check columns present in orginal df but not in df_phish
def complete_test_data(original, new_df):

    # add columns with unavailable info with -1
    missing_columns = list(original.columns.difference(new_df.columns))
    # missing_columns.remove('phishing')
    for column in missing_columns:
        new_df.loc[:,column] = -1

    # remove url, domain, directory, params, file columns
    new_df = new_df.drop(columns=['url','domain','directory','params','file'])
    
    return new_df


In [205]:
df_final = complete_test_data(origin_data,df_res)
origin_data.columns.difference(df_final.columns)

Index([], dtype='object')

In [206]:
df_final.head()

,type,length_url,domain_length,directory_length,file_length,params_length,qty_at_url,qty_questionmark_url,qty_underline_url,qty_hyphen_url,qty_equal_url,qty_dot_url,qty_hashtag_url,qty_percent_url,qty_plus_url,qty_dollar_url,qty_exclamation_url,qty_asterisk_url,qty_comma_url,qty_slash_url,qty_space_url,qty_tilde_url,qty_and_url,qty_at_domain,qty_questionmark_domain,qty_underline_domain,qty_hyphen_domain,qty_equal_domain,qty_dot_domain,qty_hashtag_domain,qty_percent_domain,qty_plus_domain,qty_dollar_domain,qty_exclamation_domain,qty_asterisk_domain,qty_comma_domain,qty_slash_domain,qty_space_domain,qty_tilde_domain,qty_and_domain,qty_at_params,qty_questionmark_params,qty_underline_params,qty_hyphen_params,qty_equal_params,qty_dot_params,qty_hashtag_params,qty_percent_params,qty_plus_params,qty_dollar_params,qty_exclamation_params,qty_asterisk_params,qty_comma_params,qty_slash_params,qty_space_params,qty_tilde_params,qty_and_params,qty_at_directory,qty_questionmark_directory,qty_underline_directory,qty_hyphen_directory,qty_equal_directory,qty_dot_directory,qty_hashtag_directory,qty_percent_directory,qty_plus_directory,qty_dollar_directory,qty_exclamation_directory,qty_asterisk_directory,qty_comma_directory,qty_slash_directory,qty_space_directory,qty_tilde_directory,qty_and_directory,qty_at_file,qty_questionmark_file,qty_underline_file,qty_hyphen_file,qty_equal_file,qty_dot_file,qty_hashtag_file,qty_percent_file,qty_plus_file,qty_dollar_file,qty_exclamation_file,qty_asterisk_file,qty_comma_file,qty_slash_file,qty_space_file,qty_tilde_file,qty_and_file,qty_vowels_domain,url_shortened,qty_params,url_google_index,domain_google_index,email_in_url,domain_spf,qty_tld_url,tld_present_params,phishing,asn_ip,domain_in_ip,qty_ip_resolved,qty_mx_servers,qty_nameservers,qty_redirects,server_client_domain,time_domain_activation,time_domain_expiration,time_response,tls_ssl_certificate,ttl_hostname
0,phishing,16,0,0,16,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1.000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,benign,35,0,17,17,0,0,0,1,0,0,2,0,0,0,0,0,0,0,2,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0.000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,benign,31,0,25,5,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0.000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,benign,118,0,18,0,26,0,0,0,16,0,2,0,0,0,0,0,0,0,5,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,1,0,0,1,1,0.000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,benign,45,0,18,26,0,0,0,1,1,0,2,0,0,0,0,0,0,0,6,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0.000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [207]:
cols = list(origin_data.columns)
df_final = df_final[cols]

In [208]:
df_final.describe()

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,qty_tilde_url,qty_comma_url,qty_plus_url,qty_asterisk_url,qty_hashtag_url,qty_dollar_url,qty_percent_url,qty_tld_url,length_url,qty_dot_domain,qty_hyphen_domain,qty_underline_domain,qty_slash_domain,qty_questionmark_domain,qty_equal_domain,qty_at_domain,qty_and_domain,qty_exclamation_domain,qty_space_domain,qty_tilde_domain,qty_comma_domain,qty_plus_domain,qty_asterisk_domain,qty_hashtag_domain,qty_dollar_domain,qty_percent_domain,qty_vowels_domain,domain_length,domain_in_ip,server_client_domain,qty_dot_directory,qty_hyphen_directory,qty_underline_directory,qty_slash_directory,qty_questionmark_directory,qty_equal_directory,qty_at_directory,qty_and_directory,qty_exclamation_directory,qty_space_directory,qty_tilde_directory,qty_comma_directory,qty_plus_directory,qty_asterisk_directory,qty_hashtag_directory,qty_dollar_directory,qty_percent_directory,directory_length,qty_dot_file,qty_hyphen_file,qty_underline_file,qty_slash_file,qty_questionmark_file,qty_equal_file,qty_at_file,qty_and_file,qty_exclamation_file,qty_space_file,qty_tilde_file,qty_comma_file,qty_plus_file,qty_asterisk_file,qty_hashtag_file,qty_dollar_file,qty_percent_file,file_length,qty_dot_params,qty_hyphen_params,qty_underline_params,qty_slash_params,qty_questionmark_params,qty_equal_params,qty_at_params,qty_and_params,qty_exclamation_params,qty_space_params,qty_tilde_params,qty_comma_params,qty_plus_params,qty_asterisk_params,qty_hashtag_params,qty_dollar_params,qty_percent_params,params_length,tld_present_params,qty_params,email_in_url,time_response,domain_spf,asn_ip,time_domain_activation,time_domain_expiration,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
count,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,631224.000,522214.000
mean,1.512,1.145,0.172,2.079,-0.036,0.074,-0.170,-0.059,-0.172,-0.172,-0.159,-0.170,-0.104,-0.172,-0.172,-0.172,0.201,0.259,45.953,-0.761,-0.892,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,-0.905,0.470,1.530,-1.000,-1.000,0.885,0.389,-0.150,0.991,-0.221,-0.214,-0.220,-0.219,-0.221,-0.221,-0.208,-0.220,-0.209,-0.221,-0.221,-0.221,-0.169,24.760,-0.052,0.214,-0.199,-0.413,-0.413,-0.411,-0.413,-0.412,-0.413,-0.413,-0.412,-0.412,-0.379,-0.413,-0.413,-0.413,-0.230,11.868,-0.808,-0.819,-0.820,-0.877,-0.877,-0.662,-0.877,-0.785,-0.877,-0.877,-0.877,-0.877,-0.846,-0.877,-0.877,-0.877,-0.615,5.985,0.998,0.214,0.000,-1.000,0.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,1.000,1.000,0.058,0.180
std,1.819,2.978,1.263,2.271,0.574,1.012,0.383,0.807,0.380,0.384,0.402,0.392,0.744,0.38

## Final comparison & adjustment with model features

In [209]:
def feature_check(model, df):
    extra_col = df.columns.difference(model.feature_names_in_)
    if not extra_col.empty:
        df = df.drop(columns=extra_col)
        df = df[model.feature_names_in_]
    return df

In [210]:
# obtain ttl for hostname of url
# import dns.resolver

# def get_ttl(url):
#     try:
#         result = dns.resolver.resolve(url, 'A')
#         if result.response.answer:
#             return int(result.response.answer[0].ttl)
#     except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN,
#             dns.exception.Timeout, dns.resolver.NoNameservers):
#         return 0
#     return 0

# df_phish.loc[:,'ttl_hostname'] = df_phish.loc[:,'domain'].apply(lambda i: get_ttl(i))

# import requests
# def check_ssl_certificate(url):
#     try:
#         response = requests.head(url)
#         # Check if the response status code is 200 (OK) or 301 (Moved Permanently)
#         if response.status_code == 200 or response.status_code == 301:
#             return 1  # SSL certificate is available
#     except requests.exceptions.SSLError:
#         pass  # SSL certificate is not available or there is an SSL error
#     except requests.exceptions.RequestException:
#         pass  # Handle other request exceptions if needed
#     return 0  # SSL certificate is not available

# df_phish.loc[:,'tls_ssl_certificate	'] = df_phish.loc[:,'url'].apply(check_ssl_certificate)

### fit trained model

In [211]:
import pickle

with open("../models/model.pkl", "rb") as f:
    model = pickle.load(f)

In [212]:
# # Load the compressed model
# with bz2.BZ2File('../model/rf_model_0327.pbz2', 'rb') as f:
#     compressed_model = f.read()

# # Decompress and load the model
# model = pickle.loads(compressed_model)

df_to_fit = feature_check(model,df_final)
# Compare predicted results with indicator column
predicted = model.predict(df_to_fit)
indicator_column = df_final['phishing'].values


In [213]:
# Step 1: Check for NaN values in 'phishing' column
nan_indices = df_final['phishing'].isnull()
if nan_indices.any():
    print("Found NaN values in 'phishing' column. Removing rows...")

# Step 2: Remove rows with NaN values from both 'phishing' and df_to_fit
df_final_clean = df_final.dropna(subset=['phishing'])
df_to_fit_clean = df_to_fit.drop(df_to_fit.index[nan_indices])

# Step 3: Make predictions and compare with 'phishing' column
predicted = model.predict(df_to_fit_clean)
indicator_column = df_final_clean['phishing'].values

# Calculate confusion matrix
confusion_mat = confusion_matrix(indicator_column, predicted)

# Calculate accuracy score
accuracy = accuracy_score(indicator_column, predicted)

# Now you can use confusion_mat and accuracy
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy Score:", accuracy)


Found NaN values in 'phishing' column. Removing rows...
Confusion Matrix:
[[145816 282287]
 [ 70798  23313]]
Accuracy Score: 0.3238691417694662


In [214]:
count_minus_one(df_to_fit)

{'qty_dot_url': 109010,
 'qty_hyphen_url': 109010,
 'qty_underline_url': 109010,
 'qty_slash_url': 109010,
 'qty_questionmark_url': 109010,
 'qty_equal_url': 109010,
 'qty_at_url': 109010,
 'qty_and_url': 109010,
 'qty_exclamation_url': 109010,
 'qty_space_url': 109010,
 'qty_tilde_url': 109010,
 'qty_comma_url': 109010,
 'qty_plus_url': 109010,
 'qty_asterisk_url': 109010,
 'qty_hashtag_url': 109010,
 'qty_dollar_url': 109010,
 'qty_percent_url': 109010,
 'qty_tld_url': 0,
 'length_url': 0,
 'qty_dot_domain': 570992,
 'qty_hyphen_domain': 570992,
 'qty_underline_domain': 570992,
 'qty_at_domain': 570992,
 'qty_vowels_domain': 0,
 'domain_length': 0,
 'domain_in_ip': 631224,
 'server_client_domain': 631224,
 'qty_dot_directory': 139300,
 'qty_hyphen_directory': 139300,
 'qty_underline_directory': 139300,
 'qty_slash_directory': 139300,
 'qty_questionmark_directory': 139300,
 'qty_equal_directory': 139300,
 'qty_at_directory': 139300,
 'qty_and_directory': 139300,
 'qty_exclamation_dire

In [215]:
# Evaluate prediction performance
# Create confusion matrix
confusion_mat = confusion_matrix(indicator_column, predicted)
# Assuming 'indicator_column' and 'predicted' are defined
accuracy = accuracy_score(indicator_column, predicted)
precision = precision_score(indicator_column, predicted)
print(confusion_mat)
print('accuracy:', accuracy)


[[145816 282287]
 [ 70798  23313]]
accuracy: 0.3238691417694662


In [216]:
confusion_mat_2 = confusion_matrix(indicator_column, predicted)
# Assuming 'indicator_column' and 'predicted' are defined
accuracy_2 = accuracy_score(indicator_column, predicted)
precision_2 = precision_score(indicator_column, predicted)
print(confusion_mat_2)
print('accuracy:', accuracy_2)

[[145816 282287]
 [ 70798  23313]]
accuracy: 0.3238691417694662


In [217]:
# check how many 0s and 1s predicted
np.unique(predicted,return_counts=True)

(array([0, 1], dtype=int64), array([216614, 305600], dtype=int64))